In [21]:
import re
import pandas as pd
from collections import defaultdict

# ---------- helpers ----------

def _cells_from_table(table):
    # Поддержка и object-таблиц SDK, и dict (если где-то приводил к dict)
    if hasattr(table, "cells"):
        return table.cells
    return table.get("cells", [])

def _get(cell, key, default=None):
    if hasattr(cell, key):
        return getattr(cell, key)
    return cell.get(key, default)

def table_to_grid(table):
    """
    Возвращает:
      grid: dict[(row_idx, col_idx)] = text
      rows: dict[row_idx] = dict[col_idx]=text
    """
    cells = _cells_from_table(table)
    grid = {}
    rows = defaultdict(dict)

    for c in cells:
        r = _get(c, "row_index", _get(c, "rowIndex", 0))
        k = _get(c, "column_index", _get(c, "columnIndex", 0))
        txt = _get(c, "content", "") or ""
        txt = str(txt).strip()

        grid[(r, k)] = txt
        rows[r][k] = txt

    return grid, rows

def find_index_row_and_cols(rows):
    """
    Ищем строку, где col0 == '1', и в ней значения '4','7','13'.
    Возвращаем (index_row, col_year, col_amount, col_code) или (None, None, None, None).
    """
    for r in sorted(rows.keys()):
        if rows[r].get(0, "").strip() != "1":
            continue

        # В этой строке ищем индексы колонок
        col_year = col_amount = col_code = None
        for c, v in rows[r].items():
            vv = str(v).strip()
            if vv == "4":
                col_year = c
            elif vv == "7":
                col_amount = c
            elif vv == "13":
                col_code = c

        if col_year is not None and col_amount is not None and col_code is not None:
            return r, col_year, col_amount, col_code

    return None, None, None, None

def extract_rows_for_df(table_idx, table, fallback_cols=None):
    """
    Делает "чистые" строки таблицы для DataFrame:
    - вырезает многоуровневую шапку (строку индексов и всё выше) если нашла
    - иначе берёт всё как есть
    - возвращает список dict-строк
    """
    _, rows = table_to_grid(table)
    idx_row, col_year, col_amount, col_code = find_index_row_and_cols(rows)

    # если в этой таблице индексы не нашли, используем fallback от table 1
    if idx_row is None:
        if not fallback_cols:
            # вообще нет понимания колонок — вернем сырье
            clean_row_ids = sorted(rows.keys())
            out = []
            for r in clean_row_ids:
                out.append({
                    "table_idx": table_idx,
                    "row_idx": r,
                    "col0": rows[r].get(0, ""),
                    "raw_row": rows[r],
                })
            return out, None
        col_year, col_amount, col_code = fallback_cols
        clean_row_ids = sorted(rows.keys())
    else:
        # берём только строки НИЖЕ строки индексов
        clean_row_ids = [r for r in sorted(rows.keys()) if r > idx_row]

    out = []
    for r in clean_row_ids:
        row = rows[r]
        out.append({
            "table_idx": table_idx,
            "row_idx": r,
            "col0": row.get(0, ""),
            "year_cell": row.get(col_year, "") if col_year is not None else "",
            "amount_cell": row.get(col_amount, "") if col_amount is not None else "",
            "code_cell": row.get(col_code, "") if col_code is not None else "",
            "raw_row": row,
        })

    return out, (col_year, col_amount, col_code)

# ---------- main logic ----------

if len(result.tables) <= 1:
    print("❌ Недостаточно таблиц!")
else:
    all_rows = []

    # 0) Таблица 0 — шапка, отбрасываем
    print("✅ Таблица 0 пропущена (шапка документа)")

    # 1) Таблица 1 — находим строку индексов, режем шапку
    t1_rows, cols = extract_rows_for_df(1, result.tables[1], fallback_cols=None)
    if cols is None:
        raise RuntimeError("Не смог найти строку индексов в Таблице 1 (col0='1' + '4','7','13').")
    print(f"✅ Таблица 1 обработана, шапка отрезана. Колонки: year={cols[0]}, amount={cols[1]}, code={cols[2]}")
    all_rows.extend(t1_rows)

    # 2+) Остальные таблицы — берём в учёт (и тоже умеем вырезать шапку если она встретится)
    for ti in range(2, len(result.tables)):
        rows_i, _ = extract_rows_for_df(ti, result.tables[ti], fallback_cols=cols)
        all_rows.extend(rows_i)
        print(f"✅ Таблица {ti} добавлена в DataFrame: строк={len(rows_i)}")

    df_raw = pd.DataFrame(all_rows)

    # --- нормализация (год/сумма/код) ---
    def parse_year(x):
        m = re.search(r"\b(20\d{2})\b", str(x))
        return m.group(1) if m else None

    def parse_amount(x):
        s = str(x).replace(" ", "").replace(",", ".")
        m = re.search(r"(\d+(?:\.\d+)?)", s)
        return float(m.group(1)) if m else None

    def parse_code_and_name(x):
        s = str(x)
        m = re.search(r"\b(\d{3})\b", s)
        code = m.group(1) if m else None
        m2 = re.search(r"\d{3}\s*-?\s*(.+)", s)
        name = m2.group(1).strip() if m2 else None
        return code, name

    df_raw["year"] = df_raw["year_cell"].apply(parse_year)
    df_raw["amount"] = df_raw["amount_cell"].apply(parse_amount)
    df_raw[["code", "name"]] = df_raw["code_cell"].apply(lambda v: pd.Series(parse_code_and_name(v)))

    # выкидываем промежуточные итоги "Всього"
    def is_total_row(raw_row):
        for v in (raw_row or {}).values():
            if "всього" in str(v).lower().strip():
                return True
        return False

    df_raw["is_total"] = df_raw["raw_row"].apply(is_total_row)
    df = df_raw[~df_raw["is_total"]].copy()

    # оставляем только нормальные строки
    df = df[df["year"].notna() & df["amount"].notna() & df["code"].notna()].copy()

    print("\n✅ DataFrame готов!")
    print("RAW:", df_raw.shape, "CLEAN:", df.shape)
    print(df.head(15)[["table_idx","row_idx","year","code","amount","name"]])

    # пример группировки
    grouped = (df.groupby(["year","code","name"], dropna=False)["amount"]
                 .sum()
                 .reset_index()
                 .sort_values(["year","code"]))

    print("\n📌 GROUPED:")
    print(grouped.head(30))

    # при желании — сохрани
    df_raw.to_csv("income_tables_raw.csv", index=False, encoding="utf-8-sig")
    df.to_csv("income_tables_clean.csv", index=False, encoding="utf-8-sig")
    grouped.to_csv("income_grouped.csv", index=False, encoding="utf-8-sig")
    print("\n📄 Сохранено: income_tables_raw.csv, income_tables_clean.csv, income_grouped.csv")


✅ Таблица 0 пропущена (шапка документа)
✅ Таблица 1 обработана, шапка отрезана. Колонки: year=3, amount=6, code=12
✅ Таблица 2 добавлена в DataFrame: строк=21

✅ DataFrame готов!
RAW: (28, 12) CLEAN: (23, 12)
    table_idx  row_idx  year code   amount  \
0           1        3  2022  101  9387.08   
1           1        4  2022  101  2620.60   
3           1        6  2023  189   245.68   
5           1        8  2024  101  2400.00   
6           1        9  2024  101  3401.62   
8           2        1  2024  126   154.71   
9           2        2  2024  101  8200.00   
10          2        3  2024  101  7028.57   
11          2        4  2024  101  1598.61   
12          2        5  2024  126    95.80   
13          2        6  2024  101  8200.00   
15          2        8  2025  101  8717.55   
16          2        9  2025  101  7790.00   
17          2       10  2025  101  7472.01   
18          2       11  2025  101  1561.90   

                                                 name 